# 모델 대상 학습 진행

기계 학습의 가장 중요한 목표는 애플리케이션에서 사용할 수 있는 예측 모델을 학습시키는 것입니다. Azure Machine Learning에서는 스크립트를 사용하여 Scikit-Learn, Tensorflow, PyTorch, SparkML 등의 일반적인 기계 학습 프레임워크를 활용하는 모델을 학습시킬 수 있습니다. 이러한 학습 스크립트를 실험으로 실행하여 메트릭 및 출력(학습된 모델 포함)을 추적할 수 있습니다.

## 작업 영역에 연결합니다.

이 Notebook의 작업을 시작하려면 먼저 작업 영역에 연결합니다.

> **참고**: Azure 구독에 인증된 세션을 아직 설정하지 않은 경우에는 링크를 클릭하고 인증 코드를 입력한 다음 Azure에 로그인하여 인증하라는 메시지가 표시됩니다.

In [ ]:
import azureml.core
from azureml.core import Workspace

# 저장된 구성 파일에서 작업 영역 로드
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

## 학습 스크립트 만들기

Python 스크립트를 사용하여 당뇨병 데이터를 기반으로 기계 학습 모델의 학습을 진행할 것입니다. 먼저 스크립트 및 데이터 파일용 폴더를 만들어 보겠습니다.

In [ ]:
import os, shutil

# 실험 파일용 폴더 만들기
training_folder = 'diabetes-training'
os.makedirs(training_folder, exist_ok=True)

# 실험 폴더에 데이터 파일 복사
shutil.copy('data/diabetes.csv', os.path.join(training_folder, "diabetes.csv"))

이제 학습 스크립트를 만들어 폴더에 저장할 준비가 되었습니다.

> **참고**: 이 코드는 스크립트를 *작성*만 하며 실행하지는 않습니다.

In [ ]:
%%writefile $training_folder/diabetes_training.py
# 라이브러리 가져오기
from azureml.core import Run
import pandas as pd
import numpy as np
import joblib
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# 실험 실행 컨텍스트 가져오기
run = Run.get_context()

# 당뇨병 데이터 세트 로드
print("Loading Data...")
diabetes = pd.read_csv('diabetes.csv')

# 기능 및 레이블 분리
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# 데이터를 학습 세트와 테스트 세트로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# 정규화 하이퍼 매개 변수 설정
reg = 0.01

# 로지스틱 회귀 모델 학습
print('Training a logistic regression model with regularization rate of', reg)
run.log('Regularization Rate',  np.float(reg))
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# 정확도 계산
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# AUC 계산
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

# outputs 폴더에 학습된 모델 저장
os.makedirs('outputs', exist_ok=True)
joblib.dump(value=model, filename='outputs/diabetes_model.pkl')

run.complete()

## 학습 스크립트를 실험으로 실행

이제 스크립트를 실험으로 실행할 수 있습니다. 기본 환경에는 **scikit-learn** 패키지가 포함되어 있지 않으므로 구성에 이 패키지를 명시적으로 추가해야 합니다. Conda 환경은 실험을 처음 실행할 때 요청하면 작성되어 같은 구성을 사용하는 이후 실행용으로 캐시됩니다. 따라서 첫 번째 실행은 시간이 좀 더 오래 걸립니다.

In [ ]:
from azureml.core import Experiment, ScriptRunConfig, Environment
from azureml.core.conda_dependencies import CondaDependencies
from azureml.widgets import RunDetails

# 실험용 Python 환경 만들기
sklearn_env = Environment("sklearn-env")

# 필수 패키지가 설치되어 있는지 확인합니다. scikit-learn 및 Azure ML defaults 패키지가 필요합니다.
packages = CondaDependencies.create(pip_packages=['scikit-learn','azureml-defaults'])
sklearn_env.python.conda_dependencies = packages

# 스크립트 구성 만들기
script_config = ScriptRunConfig(source_directory=training_folder,
                                script='diabetes_training.py',
                                environment=sklearn_env) 

# 실험 실행 제출
experiment_name = 'mslearn-train-diabetes'
experiment = Experiment(workspace=ws, name=experiment_name)
run = experiment.submit(config=script_config)

# Notebook 위젯에서 실행 중인 실험 실행 표시
RunDetails(run).show()

# 실험 실행이 완료될 때까지 차단
run.wait_for_completion()

**Run** 개체에서 메트릭과 출력을 검색할 수 있습니다.

In [ ]:
# 로깅된 메트릭 및 파일 가져오기
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

## 학습된 모델 등록

실험의 출력에는 학습된 모델 파일(**diabetes_model.pkl**)이 포함됩니다. Azure Machine Learning 작업 영역에 이 모델을 등록하면 모델 버전을 추적하고 나중에 검색할 수 있습니다.

In [ ]:
from azureml.core import Model

# 모델 등록
run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'Script'},
                   properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

# 등록된 모델 목록 표시
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

## 매개 변수가 있는 학습 스크립트 만들기

스크립트에 매개 변수를 추가하면 학습 실험을 더 유동적으로 실행할 수 있습니다. 그러면 각기 다른 설정으로 같은 학습 실험을 반복할 수 있습니다. 여기서는 모델 학습 시 로지스틱 회귀 알고리즘에 사용되는 정규화 비율의 매개 변수를 추가합니다.

이번에도 매개 변수가 있는 스크립트 및 학습 데이터용 폴더를 먼저 만들어 보겠습니다.

In [ ]:
import os, shutil

# 실험 파일용 폴더 만들기
training_folder = 'diabetes-training-params'
os.makedirs(training_folder, exist_ok=True)

# 실험 폴더에 데이터 파일 복사
shutil.copy('data/diabetes.csv', os.path.join(training_folder, "diabetes.csv"))

이제 정규화 비율 하이퍼 매개 변수에 해당하는 인수가 포함된 스크립트를 작성해 보겠습니다. Python **argparse.ArgumentParser** 개체를 사용하여 이 인수를 읽습니다.

In [ ]:
%%writefile $training_folder/diabetes_training.py
# 라이브러리 가져오기
from azureml.core import Run
import pandas as pd
import numpy as np
import joblib
import os
import argparse
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve

# 실험 실행 컨텍스트 가져오기
run = Run.get_context()

# 정규화 하이퍼 매개 변수 설정
parser = argparse.ArgumentParser()
parser.add_argument('--reg_rate', type=float, dest='reg', default=0.01)
args = parser.parse_args()
reg = args.reg

# 당뇨병 데이터 세트 로드
print("Loading Data...")
# 당뇨병 데이터 세트 로드
diabetes = pd.read_csv('diabetes.csv')

# 기능 및 레이블 분리
X, y = diabetes[['Pregnancies','PlasmaGlucose','DiastolicBloodPressure','TricepsThickness','SerumInsulin','BMI','DiabetesPedigree','Age']].values, diabetes['Diabetic'].values

# 데이터를 학습 세트와 테스트 세트로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=0)

# 로지스틱 회귀 모델 학습
print('Training a logistic regression model with regularization rate of', reg)
run.log('Regularization Rate',  np.float(reg))
model = LogisticRegression(C=1/reg, solver="liblinear").fit(X_train, y_train)

# 정확도 계산
y_hat = model.predict(X_test)
acc = np.average(y_hat == y_test)
print('Accuracy:', acc)
run.log('Accuracy', np.float(acc))

# AUC 계산
y_scores = model.predict_proba(X_test)
auc = roc_auc_score(y_test,y_scores[:,1])
print('AUC: ' + str(auc))
run.log('AUC', np.float(auc))

os.makedirs('outputs', exist_ok=True)
joblib.dump(value=model, filename='outputs/diabetes_model.pkl')

run.complete()

## 인수가 포함된 스크립트 실행

이번에도 앞에서 만들었던 환경을 재사용하여 이전과 마찬가지로 스크립트를 실험으로 실행합니다. 단, 이번에는 스크립트가 인수로 사용해야 하는 **--reg_rate** 매개 변수를 제공해야 합니다.

In [ ]:
# 스크립트 구성 만들기
script_config = ScriptRunConfig(source_directory=training_folder,
                                script='diabetes_training.py',
                                arguments = ['--reg_rate', 0.1],
                                environment=sklearn_env) 

# 실험 제출
experiment_name = 'mslearn-train-diabetes'
experiment = Experiment(workspace=ws, name=experiment_name)
run = experiment.submit(config=script_config)
RunDetails(run).show()
run.wait_for_completion()

이번에도 완료된 실행에서 메트릭과 출력을 가져올 수 있습니다.

In [ ]:
# 로깅된 메트릭 가져오기
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

## 모델의 새 버전 등록

이제 새 모델을 학습시켰으므로 작업 영역에서 해당 모델을 새 버전으로 등록할 수 있습니다.

In [ ]:
from azureml.core import Model

# 모델 등록
run.register_model(model_path='outputs/diabetes_model.pkl', model_name='diabetes_model',
                   tags={'Training context':'Parameterized script'},
                   properties={'AUC': run.get_metrics()['AUC'], 'Accuracy': run.get_metrics()['Accuracy']})

# 등록된 모델 목록 표시
for model in Model.list(ws):
    print(model.name, 'version:', model.version)
    for tag_name in model.tags:
        tag = model.tags[tag_name]
        print ('\t',tag_name, ':', tag)
    for prop_name in model.properties:
        prop = model.properties[prop_name]
        print ('\t',prop_name, ':', prop)
    print('\n')

[Azure Machine Learning Studio](https://ml.azure.com)의 **모델** 페이지에서 작업 영역에 등록된 모델을 확인할 수도 있습니다.

탐색을 완료했으면 이 Notebook을 닫고 컴퓨팅 인스턴스를 종료할 수 있습니다.